<a href="https://colab.research.google.com/github/ggwnsghgg/FastCampus/blob/master/TF_IDF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. 컨텐츠기반 추천시스템
2. 컨텐츠를 활용할 수 있는지
3. 어떻게 컨텐츠들이 서로 연관이 있는지 평가하는 방법
4. 근접이웃기반 컨텐츠기반 추천시스템 -> k-nearest neighbor
5. naive bayes classifer -> 베이즈 추천시스템
6. TF-IDF 개념설명 그리고 실습

추천시스템 성능 평가 -> RMSE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:

path = '/content/drive/MyDrive/data/movielens'


ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [ ]:
book_df

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500.0,43.0,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220.0,13.0,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200.0,54.0,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500.0,31.0,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
4,6,라플라스의 마녀,히가시노 게이고,현대문학,2016.01,13320.0,154.0,8.8,"일본 추리소설계를 대표하는 최고의 베스트셀러 작가 히가시노 게이고가 2015년, 데...",http://image.yes24.com/goods/23630397/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
77128,79771,하루만 사랑해 2,NaN,파피루스(디앤씨미디어),2003.06,8100.0,0.0,0.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375484/L,로맨스
77129,79772,서울 여자와 대구 남자가 만날 때,정민,영상노트,2003.06,8100.0,0.0,0.0,NaN,http://image.yes24.com/goods/376458/L,로맨스
77130,79773,kiss me...,NaN,현무,2003.06,7200.0,2.0,5.0,NaN,http://image.yes24.com/goods/377351/L,로맨스
77131,79774,어린 엄마 2,임은희,반디출판사,2003.07,8100.0,0.0,0.0,NaN,http://image.yes24.com/goods/377607/L,로맨스


In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [8]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [9]:
total_count = len(movies_df.index)
total_genres = list(set([genre for sublist in list(map(lambda x: x.split('|'), movies_df['genres'])) for genre in sublist]))

In [11]:
print(f"전체 영화 수: {total_count}")
print(f"장르: {total_genres}")

전체 영화 수: 9742
장르: ['Romance', '(no genres listed)', 'Mystery', 'War', 'Animation', 'Drama', 'Sci-Fi', 'Western', 'Children', 'Crime', 'Comedy', 'Adventure', 'Horror', 'IMAX', 'Film-Noir', 'Documentary', 'Thriller', 'Action', 'Musical', 'Fantasy']


In [12]:

print(len(total_genres))

20


In [13]:
genre_count = dict.fromkeys(total_genres)

for each_genre_list in movies_df['genres']:
    for genre in each_genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else:
            genre_count[genre] = genre_count[genre]+1